In [1]:
!uv pip install torch
!uv pip install torchvision
!uv pip install tqdm

Using Python 3.12.7 environment at: /app/.venv
Resolved 10 packages in 186ms                                        
⠙ Preparing packages... (0/7)                                                   
⠙ Preparing packages... (0/7)----     0 B/175.43 KiB                    
⠙ Preparing packages... (0/7)---- 14.88 KiB/175.43 KiB                  
filelock   ------------------------------     0 B/15.78 KiB
⠙ Preparing packages... (0/7)---- 14.88 KiB/175.43 KiB                  
filelock   ------------------------------     0 B/15.78 KiB
fsspec     ------------------------------ 14.88 KiB/175.43 KiB
⠙ Preparing packages... (0/7)----     0 B/1.64 MiB                      
filelock   ------------------------------     0 B/15.78 KiB
fsspec     ------------------------------ 14.88 KiB/175.43 KiB
⠙ Preparing packages... (0/7)---- 14.88 KiB/1.64 MiB                    
filelock   ------------------------------ 15.78 KiB/15.78 KiB
fsspec     ------------------------------ 14.88 KiB/175.43 KiB
⠙ Prep

In [3]:
import torch
from torchvision import datasets, transforms

# バッチサイズ
BATCH_SIZE = 64

# 画像データの変換方法指定
transform = transforms.Compose([
    transforms.ToTensor(),  # テンソルに変換＆0-255を0-1に正規化
])

# MNISTのデータセットをダウンロード
train_dataset = datasets.MNIST(
    root = "./data", # データの保存場所
    train = True, # 訓練データを取得する
    download = True, # ダウンロードする
    transform = transform # 画像データの変換方法指定
)

# テスト用のデータセットをダウンロード
test_dataset = datasets.MNIST(
    root = "./data", # データの保存場所
    train = False, # テストデータを取得する
    download = True, # ダウンロードする
    transform = transform # 画像データの変換方法指定
)

# データローダーの作成
train_loader = torch.utils.data.DataLoader(
    train_dataset, # データセットの指定
    batch_size = BATCH_SIZE, # バッチサイズの指定
    shuffle = True # シャッフルする
)

test_loader = torch.utils.data.DataLoader(
    test_dataset, # データセットの指定
    batch_size = BATCH_SIZE, # バッチサイズの指定
    shuffle = True # シャッフルしない
)

# 入力データのshapeと中身を確認
print(train_dataset[0][0].shape)
print(train_dataset[0][0])

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:09<00:00, 1.01MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 164kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:02<00:00, 571kB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 1.11MB/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

torch.Size([1, 28, 28])
tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000

In [11]:
# モデルの作成

import torch.nn as nn

class MNISTModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(28*28, 512),
            # nn.ReLU(),
            nn.Linear(512,256),
            # nn.ReLU(),
            nn.Linear(256,10),
            nn.LogSoftmax(dim=1),
        )
    
    def forward(self, x: torch.Tensor):
        x = x.view(-1, 28*28)
        x = self.model(x)
        return x

In [12]:
from tqdm import tqdm

model = MNISTModel()

criterion = nn.NLLLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

for epoch in range(10):
    total_loss = 0
    for images, labels in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    # lossは平均を取って表示
    print(f'Epoch: {epoch+1}, Loss: {total_loss / len(train_loader)}')

100%|██████████| 938/938 [00:03<00:00, 289.15it/s]


Epoch: 1, Loss: 0.5236199878092641


100%|██████████| 938/938 [00:02<00:00, 328.35it/s]


Epoch: 2, Loss: 0.438464221058052


100%|██████████| 938/938 [00:02<00:00, 324.27it/s]


...

Epoch: 8, Loss: 0.511528052262533


100%|██████████| 938/938 [00:03<00:00, 307.11it/s]


Epoch: 9, Loss: 0.374437024939988


100%|██████████| 938/938 [00:02<00:00, 315.16it/s]

Epoch: 10, Loss: 0.3868189398914194


In [13]:
# modelの評価
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'accuracy: {100 * correct /total}%')

accuracy: 85.7%
